# Warehouse Digital Twin - Anomaly Detection Analysis

This notebook demonstrates the machine learning approach for anomaly detection in warehouse sensor data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import joblib
import os

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

## 1. Generate Sample Sensor Data for Testing

First, let's generate some synthetic sensor data to test our anomaly detection algorithm.

In [ ]:
def generate_sample_data(sensor_type, num_samples=1000, anomaly_percentage=5):
    """
    Generate synthetic sensor data with anomalies
    
    Parameters:
    - sensor_type: Type of sensor (temperature, humidity, etc.)
    - num_samples: Number of data points to generate
    - anomaly_percentage: Percentage of data that should be anomalous
    """
    # Define normal ranges for different sensor types
    normal_ranges = {
        'temperature': {'mean': 25, 'std': 2.5, 'anomaly_factor': 3.5},
        'humidity': {'mean': 50, 'std': 8, 'anomaly_factor': 3},
        'pressure': {'mean': 1013, 'std': 5, 'anomaly_factor': 4},
        'weight': {'mean': 2500, 'std': 500, 'anomaly_factor': 2.5},
        'battery_level': {'mean': 75, 'std': 15, 'anomaly_factor': 3}
    }
    
    if sensor_type not in normal_ranges:
        raise ValueError(f"Sensor type {sensor_type} not supported")
        
    # Get parameters for this sensor type
    params = normal_ranges[sensor_type]
    
    # Generate normal data
    normal_count = int(num_samples * (100 - anomaly_percentage) / 100)
    anomaly_count = num_samples - normal_count
    
    normal_data = np.random.normal(
        params['mean'], 
        params['std'], 
        normal_count
    )
    
    # Generate anomaly data (outside normal range)
    anomaly_data = []
    for _ in range(anomaly_count):
        # Randomly choose high or low anomaly
        if np.random.random() > 0.5:
            # High anomaly
            value = params['mean'] + (params['std'] * params['anomaly_factor'] * np.random.random())
        else:
            # Low anomaly
            value = params['mean'] - (params['std'] * params['anomaly_factor'] * np.random.random())
        anomaly_data.append(value)
    
    # Combine data and create labels (1 for normal, -1 for anomaly)
    all_values = list(normal_data) + anomaly_data
    all_labels = [1] * normal_count + [-1] * anomaly_count
    
    # Create timestamps (last 1000 minutes, one reading per minute)
    timestamps = pd.date_range(
        end=pd.Timestamp.now(), 
        periods=num_samples, 
        freq='T'
    )
    
    # Create a dataframe
    df = pd.DataFrame({
        'timestamp': timestamps,
        'value': all_values,
        'anomaly': all_labels
    })
    
    # Shuffle the dataframe to mix normal and anomaly data
    df = df.sample(frac=1).reset_index(drop=True)
    
    return df

In [ ]:
# Generate sample data for a few sensor types
sensor_types = ['temperature', 'humidity', 'pressure']
sample_data = {}

for sensor_type in sensor_types:
    sample_data[sensor_type] = generate_sample_data(sensor_type, num_samples=1000, anomaly_percentage=5)
    print(f"Generated {len(sample_data[sensor_type])} samples for {sensor_type} sensor")

Let's visualize the sample temperature data to see what we're working with:

In [ ]:
def plot_sensor_data(df, sensor_type):
    plt.figure(figsize=(14, 7))
    
    # Plot all data points
    plt.scatter(
        df['timestamp'], 
        df['value'],
        c=df['anomaly'].map({1: 'blue', -1: 'red'}),
        alpha=0.6
    )
    
    plt.title(f"{sensor_type} Sensor Readings with Anomalies")
    plt.xlabel("Time")
    plt.ylabel(f"{sensor_type.capitalize()} Value")
    plt.grid(True)
    
    # Add legend
    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=10, label='Normal'),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label='Anomaly')
    ]
    plt.legend(handles=legend_elements)
    
    plt.tight_layout()
    plt.show()
    
    # Also show a histogram of values
    plt.figure(figsize=(14, 5))
    
    # Plot histograms for normal and anomaly data
    sns.histplot(
        df[df['anomaly'] == 1]['value'], 
        color='blue', 
        label='Normal',
        alpha=0.6
    )
    sns.histplot(
        df[df['anomaly'] == -1]['value'], 
        color='red', 
        label='Anomaly',
        alpha=0.6
    )
    
    plt.title(f"Distribution of {sensor_type.capitalize()} Values")
    plt.xlabel(f"{sensor_type.capitalize()} Value")
    plt.ylabel("Count")
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot temperature data
plot_sensor_data(sample_data['temperature'], 'temperature')

## 2. Implementing Anomaly Detection with Isolation Forest

Now let's implement the Isolation Forest algorithm for anomaly detection:

In [ ]:
def train_isolation_forest(data, contamination=0.05):
    """
    Train an Isolation Forest model for anomaly detection
    
    Parameters:
    - data: DataFrame with sensor readings
    - contamination: Expected proportion of anomalies
    
    Returns:
    - Trained model and scaler
    """
    # Extract the values and reshape for sklearn
    X = data['value'].values.reshape(-1, 1)
    
    # Scale the data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Train Isolation Forest
    model = IsolationForest(
        n_estimators=100,
        max_samples='auto',
        contamination=contamination,
        random_state=42
    )
    model.fit(X_scaled)
    
    return model, scaler

In [ ]:
def evaluate_model(model, scaler, data):
    """
    Evaluate model performance and compare with ground truth
    """
    # Prepare data
    X = data['value'].values.reshape(-1, 1)
    X_scaled = scaler.transform(X)
    
    # Predict anomalies
    predictions = model.predict(X_scaled)
    scores = model.score_samples(X_scaled)
    
    # Add predictions to dataframe
    results = data.copy()
    results['predicted'] = predictions
    results['anomaly_score'] = scores
    
    # Calculate metrics
    from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
    
    cm = confusion_matrix(results['anomaly'], results['predicted'])
    report = classification_report(results['anomaly'], results['predicted'])
    accuracy = accuracy_score(results['anomaly'], results['predicted'])
    
    print(f"Accuracy: {accuracy:.4f}")
    print("\nConfusion Matrix:")
    print(cm)
    print("\nClassification Report:")
    print(report)
    
    # Visualize results
    plt.figure(figsize=(14, 7))
    
    # Plot actual anomalies
    plt.scatter(
        results['timestamp'],
        results['value'],
        c=results['anomaly'].map({1: 'blue', -1: 'red'}),
        alpha=0.3,
        label='Actual'
    )
    
    # Plot predicted anomalies as circles
    anomaly_indices = results[results['predicted'] == -1].index
    plt.scatter(
        results.iloc[anomaly_indices]['timestamp'],
        results.iloc[anomaly_indices]['value'],
        edgecolors='green',
        facecolors='none',
        s=80,
        label='Predicted Anomaly'
    )
    
    plt.title("Actual vs Predicted Anomalies")
    plt.xlabel("Time")
    plt.ylabel("Sensor Value")
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    # Plot anomaly scores
    plt.figure(figsize=(14, 5))
    plt.plot(results['timestamp'], results['anomaly_score'])
    
    # Highlight anomalies
    plt.scatter(
        results.iloc[anomaly_indices]['timestamp'],
        results.iloc[anomaly_indices]['anomaly_score'],
        color='red',
        label='Anomaly'
    )
    
    # Draw threshold line
    threshold = model.threshold_
    plt.axhline(y=threshold, color='r', linestyle='--', label=f'Threshold: {threshold:.4f}')
    
    plt.title("Anomaly Scores Over Time")
    plt.xlabel("Time")
    plt.ylabel("Anomaly Score")
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    return results

In [ ]:
# Train and evaluate model for temperature
temp_model, temp_scaler = train_isolation_forest(sample_data['temperature'], contamination=0.05)
temp_results = evaluate_model(temp_model, temp_scaler, sample_data['temperature'])

## 3. Implementing Sliding Window Anomaly Detection

A more realistic approach for streaming data is to use a sliding window technique:

In [ ]:
def simulate_streaming_detection(data, window_size=100, step=10):
    """
    Simulate detecting anomalies in streaming data using a sliding window
    """
    sorted_data = data.sort_values('timestamp').reset_index(drop=True)
    results = pd.DataFrame()
    
    # Process data in chunks
    for start_idx in range(0, len(sorted_data) - window_size, step):
        end_idx = start_idx + window_size
        
        # Training window
        train_window = sorted_data.iloc[start_idx:end_idx]
        
        # Next batch to predict
        predict_window = sorted_data.iloc[end_idx:end_idx+step]
        
        if len(predict_window) == 0:
            break
            
        # Train model on window
        X_train = train_window['value'].values.reshape(-1, 1)
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        
        model = IsolationForest(
            n_estimators=100,
            max_samples='auto',
            contamination=0.05,
            random_state=42
        )
        model.fit(X_train_scaled)
        
        # Predict on next batch
        X_pred = predict_window['value'].values.reshape(-1, 1)
        X_pred_scaled = scaler.transform(X_pred)
        
        predictions = model.predict(X_pred_scaled)
        scores = model.score_samples(X_pred_scaled)
        
        # Store results
        batch_results = predict_window.copy()
        batch_results['predicted'] = predictions
        batch_results['anomaly_score'] = scores
        batch_results['window_start'] = start_idx
        batch_results['window_end'] = end_idx
        
        results = pd.concat([results, batch_results])
    
    return results

In [ ]:
# Simulate streaming detection
streaming_results = simulate_streaming_detection(sample_data['temperature'], window_size=100, step=25)

# Visualize results
plt.figure(figsize=(14, 7))

# Plot actual values
plt.scatter(
    streaming_results['timestamp'],
    streaming_results['value'],
    c=streaming_results['anomaly'].map({1: 'blue', -1: 'red'}),
    alpha=0.3
)

# Plot predicted anomalies
anomaly_indices = streaming_results[streaming_results['predicted'] == -1].index
plt.scatter(
    streaming_results.iloc[anomaly_indices]['timestamp'],
    streaming_results.iloc[anomaly_indices]['value'],
    edgecolors='green',
    facecolors='none',
    s=80,
    label='Predicted Anomaly'
)

plt.title("Streaming Anomaly Detection Results")
plt.xlabel("Time")
plt.ylabel("Temperature")
plt.legend()
plt.tight_layout()
plt.show()

# Show metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

print("Streaming Detection Results:")
print(f"Accuracy: {accuracy_score(streaming_results['anomaly'], streaming_results['predicted']):.4f}")
print("\nConfusion Matrix:")
print(confusion_matrix(streaming_results['anomaly'], streaming_results['predicted']))
print("\nClassification Report:")
print(classification_report(streaming_results['anomaly'], streaming_results['predicted']))

## 4. Saving and Loading Models

Let's demonstrate how to save and load models for production use:

In [ ]:
# Create models directory if it doesn't exist
os.makedirs('models', exist_ok=True)

# Save models for different sensor types
for sensor_type in sensor_types:
    # Train model
    model, scaler = train_isolation_forest(sample_data[sensor_type])
    
    # Save model and scaler
    joblib.dump(model, f'models/{sensor_type}_model.pkl')
    joblib.dump(scaler, f'models/{sensor_type}_scaler.pkl')
    
    print(f"Saved model for {sensor_type}")
    
# Test loading the model
loaded_model = joblib.load('models/temperature_model.pkl')
loaded_scaler = joblib.load('models/temperature_scaler.pkl')

print("\nModel loaded successfully")

# Test prediction with loaded model
test_sample = np.array([[35.0]])  # An anomalous temperature
test_scaled = loaded_scaler.transform(test_sample)
prediction = loaded_model.predict(test_scaled)[0]
score = loaded_model.score_samples(test_scaled)[0]

print(f"Test prediction for temperature=35.0: {'Anomaly' if prediction == -1 else 'Normal'} (score: {score:.4f})")

## 5. Integration with the Digital Twin System

The ML models trained here can be integrated with the Warehouse Digital Twin system in the following ways:

1. **Offline Training:** Use this notebook to train initial models with historical data
2. **Online Learning:** The `ml_detector.py` script performs online learning and adaptation
3. **Real-time Anomaly Detection:** Anomalies are published to a dedicated Kafka topic
4. **Dashboard Integration:** The dashboard can be enhanced to display ML-detected anomalies

The key advantage of using machine learning for anomaly detection is that it can adapt to the normal patterns of the warehouse and detect subtle anomalies that rule-based systems might miss.

In [ ]:
# Summary of model performance for different sensor types
performance_summary = {}

for sensor_type in sensor_types:
    print(f"\n==== Evaluating {sensor_type.upper()} sensor ====\n")
    model, scaler = train_isolation_forest(sample_data[sensor_type])
    results = evaluate_model(model, scaler, sample_data[sensor_type])
    
    # Calculate performance metrics
    accuracy = accuracy_score(results['anomaly'], results['predicted'])
    cm = confusion_matrix(results['anomaly'], results['predicted'])
    
    # True positives, false positives, etc.
    tp = cm[1, 1] if cm.shape == (2, 2) else 0
    fp = cm[0, 1] if cm.shape == (2, 2) else 0
    tn = cm[0, 0] if cm.shape == (2, 2) else 0
    fn = cm[1, 0] if cm.shape == (2, 2) else 0
    
    # Detection rate and false alarm rate
    detection_rate = tp / (tp + fn) if (tp + fn) > 0 else 0
    false_alarm_rate = fp / (fp + tn) if (fp + tn) > 0 else 0
    
    performance_summary[sensor_type] = {
        'accuracy': accuracy,
        'detection_rate': detection_rate,
        'false_alarm_rate': false_alarm_rate
    }

# Create performance summary table
summary_df = pd.DataFrame(performance_summary).T
summary_df = summary_df * 100  # Convert to percentages
summary_df = summary_df.round(2)

print("\n===== Performance Summary =====\n")
print(summary_df)